In [8]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import pandas as pd
from helper import *
pd.options.display.max_columns = None

## Import DataSet

In [90]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
housing_price_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAAVLZzU4E7ro0BiRzPG3pP8a/House%20Prices?dl=1"
census_data_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AADfweTUjMlkayMQ6JvKUOGia/Census-Income%20(KDD)%20Data%20Set?dl=0"
all_urls = [sumdata_url, housing_price_url, census_data_url]

In [91]:
get_data(all_urls) # retrieves the data if there is no data folder

In [11]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv"
housing_price_path = "data/housing dataset.csv" # has more than 30 features
census_path = "data/housing dataset.csv"
titanic_path = "data/"
# need one more
# what a brilliant idea to name files with space

## Load datasets sum_noise

In [13]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")
sumdata_noise

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,1,66957,74432,96087,103120,64272,150633,181787,180349,216912,304071,1434819,Very Large Number
1,2,96030,86875,108299,148025,16965,253819,258672,268851,404599,543092,2148748,Very Large Number
2,3,26212,23398,27668,39678,23062,65873,65660,68508,82617,115418,476405,Very Large Number
3,4,28363,33381,42447,35270,8980,52885,79144,85741,86806,147368,635169,Very Large Number
4,5,38960,50255,79879,91885,64037,127193,115760,174069,184805,250659,1221471,Very Large Number
5,6,34616,45307,50710,58657,58330,79111,99885,111644,140952,210654,794117,Very Large Number
6,7,12888,20126,22753,23219,1766,35151,40627,43459,65339,78915,363026,Large Number
7,8,30922,38291,43157,53434,33790,95398,109501,114824,168799,160433,813129,Very Large Number
8,9,33869,47235,53800,69464,35334,84096,119056,141564,137810,151482,914668,Very Large Number
9,10,28247,24863,41941,38753,101615,56320,80638,103624,132245,149352,696654,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [14]:
# Remove 'Instance' as it simply represents the row number
sumdata_noise.drop('Instance', axis = 1)

# Use 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise['Noisy Target'].values.reshape(-1, 1)

# Use 'Nosiy Target Class' as regression target
sumdata_noise_classif_Y = sumdata_noise['Noisy Target Class']

# Use rest columns as explananatory variables
sumdata_noise_X = sumdata_noise.iloc[:, 0:-2]

# Apply Feature Scaling to the dataset 
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
scY = StandardScaler()
sumdata_noise_X = scX.fit_transform(sumdata_noise_X)
sumdata_noise_reg_Y = scY.fit_transform(sumdata_noise_reg_Y)

sumdata_noise_X

/Users/kiyim/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.73204902,  0.57148372,  0.40463575, ...,  0.01029815,
         0.01473561,  0.29953474],
       [-1.73204544,  1.55295829,  0.75447754, ...,  0.8441455 ,
         1.48881339,  1.86218921],
       [-1.73204186, -0.80402556, -1.03021326, ..., -1.04344437,
        -1.04000593, -0.93382741],
       ..., 
       [ 1.73204186,  0.03042869,  0.34654902, ...,  0.76615191,
         0.79476291,  0.37581042],
       [ 1.73204544,  0.73163628,  0.32251022, ...,  0.86467558,
         1.10936719,  0.36614112],
       [ 1.73204902, -0.98058498, -1.05430829, ..., -1.14767765,
        -1.17865895, -1.17127118]])

## Load datasets sumdata

In [15]:
sumdata = pd.read_csv(sumdata_path, delimiter=";")
sumdata.head(n=1)

,Instance,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Target,Target Class
0,1,57326,68791,82549,99059,72624,142645,171174,205409,246491,295789,1073444,Very Large Number


## Preprocess sumdata dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest of the columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [16]:
# Remove 'Instance' as it simply represents the row number
sumdata.drop('Instance', axis = 1)

# Use 'Nosiy Target' as regression target
sumdata_reg_Y = sumdata['Target'].reshape(-1, 1)

# Use 'Nosiy Target Class' as classification target
sumdata_classif_Y = sumdata['Target Class']

# Use rest columns as explananatory variables
sumdata_X = sumdata.iloc[:, 0:-2]

# Apply Feature Scaling to the dataset 
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
scY = StandardScaler()
sumdata_X = scX.fit_transform(sumdata_X)
sumdata_reg_Y = scY.fit_transform(sumdata_reg_Y)

sumdata_X

/Users/kiyim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
/Users/kiyim/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.73204902,  0.25276029,  0.25275451, ...,  0.25275618,
         0.25275779,  0.25275645],
       [-1.73204544,  1.31055785,  1.31056363, ...,  1.31055123,
         1.31055284,  1.31055016],
       [-1.73204186, -0.91146488, -0.91147066, ..., -0.91146134,
        -0.91146456, -0.91146591],
       ..., 
       [ 1.73204186,  0.46131959,  0.46131382, ...,  0.46131792,
         0.46131792,  0.46131657],
       [ 1.73204544,  0.79373264,  0.79372687, ...,  0.79372498,
         0.79372497,  0.79372229],
       [ 1.73204902, -1.09338264, -1.09338265, ..., -1.09338604,
        -1.09338765, -1.09339034]])

## Load housing price dataset

In [74]:
housing_price = pd.read_csv(housing_price_path)
housing_price = housing_price#
housing_price.head(n=5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Preprocess housing price dataset

- Remove 'Id' as it simply represents the row number
- Use 'SalePrice' as regression target
- Use 'SaleCondition' as classification target
- For explananatory variables, use all the numerical variable and use the following categorical variables
    - Neighborhood, as the career of the neighbour impacts on factors such as the consuming power of the area 
    - HouseStyle, well-designed house is more expensive
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [81]:
# Remove 'Instance' as it simply represents the row number
housing_price.drop('Id', axis = 1)

# Use 'Nosiy Target' as regression target
housing_price_reg_Y = [housing_price['SalePrice']]

# Use 'Nosiy Target Class' as classification target
housing_price_classif_Y = housing_price['SaleCondition']

# Get all the numeric variables
housing_price_X = housing_price.iloc[:, 0:-2]._get_numeric_data()

# Get the two chosen categorical variables and add them to
housing_price_X = pd.concat([housing_price_X, pd.get_dummies(housing_price['Neighborhood']).iloc[:, :-1]], axis=1)
housing_price_X = pd.concat([housing_price_X, pd.get_dummies(housing_price['HouseStyle']).iloc[:, :-1]], axis=1)
housing_price_X = housing_price_X.dropna()
housing_price_X

# Apply Feature Scaling to the numeric variables and regression target 
scX = StandardScaler()
scY = StandardScaler()
housing_price_X = scX.fit_transform(housing_price_X)
housing_price_reg_Y = scY.fit_transform(housing_price_reg_Y)
housing_price_classif_Y = housing_price_classif_Y.values



# Fits Algorithms to datasets

In [102]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

In [123]:
def model( X, y, dataset_name, algorithm=LinearRegression): 
    
    algo_name = algorithm.__name__
    for chunk in data_chunks:
        
        if chunk > X.shape[0]: # if chunk is greater than the no. of examples
            break

        X = X[:, 0:chunk]
        y = y[:, 0:chunk]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        lm = algorithm()
        model = lm.fit(X_train, y_train)
        
        predictions = model.predict(X_test)
        # Validate the model
#         kFoldModel()
        #print(predictions)
        
        error = mean_squared_error(y_test, predictions)**0.5  # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
        print ("Data set: {} Algorithm: {}  Chunk Size: {} Error: {}".format(dataset_name,
                                                                          algo_name,
                                                                          chunk,
                                                                           error
                                                                         ))
              
         
    

In [124]:
def kFoldModel (X, y, data_set, kfolds, algorithm=LinearRegression):
    
    algo_name = algorithm.__name__
    
    kf = KFold(n_splits=kfolds)
    
    for train, test in kf.split(data_set):
        print("%s %s" % (train, test))

## Fits Regression Algorithms to datasets

    - Linear Regression
    - Random Forest Regression
    

In [125]:
from sklearn.ensemble import RandomForestRegressor

model(sumdata_noise_X, sumdata_noise_reg_Y,"The Sum Dataset(without noise)", LinearRegression )
sumdata_noise_X.shape
# model(sumdata_X, sumdata_reg_Y,"The Sum Dataset(without noise)", LinearRegression )
# model(housing_price_X, housing_price_reg_Y,"The Sum Dataset(without noise)", LinearRegression )

# model(sumdata_noise_X, sumdata_noise_reg_Y,"The Sum Dataset(with noise)", RandomForestRegressor )
# model(sumdata_X, sumdata_reg_Y, "The Sum Dataset(with noise)", RandomForestRegressor )
# model(housing_price_X, housing_price_reg_Y, "The Sum Dataset(with noise)", RandomForestRegressor )

Data set: The Sum Dataset(without noise) Algorithm: LinearRegression  Chunk Size: 100 Error: 0.11468765043817408
Data set: The Sum Dataset(without noise) Algorithm: LinearRegression  Chunk Size: 500 Error: 0.11481809958377741
Data set: The Sum Dataset(without noise) Algorithm: LinearRegression  Chunk Size: 1000 Error: 0.11493496516094923
Data set: The Sum Dataset(without noise) Algorithm: LinearRegression  Chunk Size: 5000 Error: 0.11464165445529044
Data set: The Sum Dataset(without noise) Algorithm: LinearRegression  Chunk Size: 10000 Error: 0.11481578692945582
Data set: The Sum Dataset(without noise) Algorithm: LinearRegression  Chunk Size: 50000 Error: 0.11433754847630337
Data set: The Sum Dataset(without noise) Algorithm: LinearRegression  Chunk Size: 100000 Error: 0.1147897849936184
Data set: The Sum Dataset(without noise) Algorithm: LinearRegression  Chunk Size: 500000 Error: 0.115041819870308


(968135, 11)

In [ ]:
model(sumdata_X, sumdata_y,"Sum without noise", LinearRegression )

# DID do cross validation

In [15]:
kFoldModel(sumdata_noise_X, sumdata_noise_reg_Y, "Noisy Sum", 2, LinearRegression )

[5 6 7 8] [0 1 2 3 4]
[0 1 2 3 4] [5 6 7 8]
